In [1]:
import cv2
import math
import time
import queue
import threading
import os

from utils.thresholding import *
from pynq import Overlay
from pynq import MMIO
import time
import matplotlib.pyplot as plt
%matplotlib inline

os.makedirs("test_images", exist_ok=True)

In [2]:
def set_bit(value, bit):
    return value | (1 << bit)

In [3]:
# Helper to extract timer registers
def extract_timer_regs(timer_name):
    regs = motor_control.ip_dict[timer_name]['registers']
    return {
        'TCSR0_addr': regs['TCSR0']['address_offset'],
        'TCSR1_addr': regs['TCSR1']['address_offset'],
        'TCSR0_fields': regs['TCSR0']['fields'],
        'TCSR1_fields': regs['TCSR1']['fields'],
        'TLR0_addr': regs['TLR0']['address_offset'],
        'TLR1_addr': regs['TLR1']['address_offset'],
    }

In [4]:
# Configure control register to enable timer in PWM mode
def build_control_value(fields, reg_fields):
    val = 0
    for field in fields:
        val = set_bit(val, reg_fields[field]['bit_offset'])
    return val

In [5]:
def control_motors(motor_en, l_forward, r_forward):
    gpio_val = 0

    # Set left motor bits
    gpio_val |= 0x4 if l_forward else 0x8

    # Set right motor bits
    gpio_val |= 0x1 if r_forward else 0x2

    # Write combined value once
    motor_en.write(0x0, gpio_val)

In [6]:
def compute_lane_heading_angle(left_fit=None, right_fit=None, y_eval=200):
    angles = []

    def angle_from_fit(fit):
        # Derivative of the 2nd-order polynomial at y = y_eval
        dy = 1.0  # pixel change in y
        dx = 2 * fit[0] * y_eval + fit[1]  # derivative at y_eval
        angle_rad = np.arctan(dx)  # slope to angle
        return -np.degrees(angle_rad)

    if left_fit is not None:
        angles.append(angle_from_fit(left_fit))
    if right_fit is not None:
        angles.append(angle_from_fit(right_fit))

    if angles:
        print(angles)
        # Average heading angle from both lanes
        return np.mean(angles)
    else:
        return None

In [7]:
def detect_lane_lines_connected_components(binary_warped):
    # Ensure the input is binary (0 and 255)
    binary = np.uint8(binary_warped * 255) if binary_warped.max() <= 1 else np.uint8(binary_warped)

    # Apply connected components
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary, connectivity=8)

    # Image dimensions
    height, width = binary.shape

    # Store points
    leftx, lefty, rightx, righty = [], [], [], []

    for i in range(1, num_labels):  # Label 0 is background
        x, y, w, h, area = stats[i]
        cx, cy = centroids[i]

        # Heuristic filters for likely lane lines
        if area > 1000 and h > 30:
            if cx < width // 2:
                coords = np.column_stack(np.where(labels == i))
                for pt in coords:
                    lefty.append(pt[0])
                    leftx.append(pt[1])
            else:
                coords = np.column_stack(np.where(labels == i))
                for pt in coords:
                    righty.append(pt[0])
                    rightx.append(pt[1])

    # Convert lists to numpy arrays
    leftx = np.array(leftx)
    lefty = np.array(lefty)
    rightx = np.array(rightx)
    righty = np.array(righty)

    return leftx, lefty, rightx, righty

In [ ]:
WIDTH = 640
HEIGHT = 480

#BOT_CROP = 40 #when avg speed is 30
#BOT_CROP = 20 #when avg speed is 20
BOT_CROP = 0

frame_in_w = 640
frame_in_h = 480

ym_per_pix = 0.1524 / 72.0
xm_per_pix = 0.2286 / 600.0
#y_eval = 210
#y_eval = 188 #Where you look for turns on y-axis of image
y_eval = 190
midx = 320

#y_bottom = HEIGHT
y_bottom = HEIGHT - BOT_CROP
y_top = int(HEIGHT * 0.4)
#y_top = 0 

angle = 0
kernel = np.ones((3, 3), np.uint8)  # Try 5x5 instead of 3x3

In [9]:
scale_factor_h = frame_in_w / 1280
scale_factor_v = frame_in_h / 720
offset = 200 * scale_factor_h
src = np.float32([
    [10, 300],
    [630, 300],
    [640, 410],
    [0, 410]
])

dst = np.float32([[offset, 0], [frame_in_w - offset, 0], [frame_in_w - offset, frame_in_h], [offset, frame_in_h]])
M = cv2.getPerspectiveTransform(src,dst)

In [10]:
def capture_frames(videoIn, frame_queue):
    while True:
        ret, frame = videoIn.read()
        if not ret:
            print("Failed to grab frame.")
            break
        if frame_queue.full():
            # If the queue is full (only 1 frame), remove the oldest frame
            frame_queue.get()  # Discard the old frame to keep the queue size at 1
        frame_queue.put(frame)

In [11]:
# Load your custom overlay
motor_control = Overlay("motor_control.bit")

# PWM SETUP (Using MMIO).....
right_motor = motor_control.axi_timer_0 #PMODB P3 for right PWM
left_motor = motor_control.axi_timer_1  #PMODB P5 for left PWM

# extract register addresses (will be the same for every Axi Timer)
R_TCSR0 = motor_control.ip_dict['axi_timer_0']['registers']['TCSR0']
R_TCSR1 = motor_control.ip_dict['axi_timer_0']['registers']['TCSR1']
R_TCSR0_address = R_TCSR0['address_offset']
R_TCSR1_address = R_TCSR1['address_offset']
R_TCSR0_register = R_TCSR0['fields'] # bit_offset for address
R_TCSR1_register = R_TCSR1['fields']
R_TLR0 = motor_control.ip_dict['axi_timer_0']['registers']['TLR0']
R_TLR1 = motor_control.ip_dict['axi_timer_0']['registers']['TLR1']
R_TLR0_address = R_TLR0['address_offset']
R_TLR1_address = R_TLR1['address_offset']

# create the configuration values for the control register
R_temp_val_0 = 0
R_temp_val_1 = 0

# The PWMA0 bit in TCSR0 and PWMB0 bit in TCSR1 must be set to 1 to enable PWM mode.
R__temp_val_0 = set_bit(R_temp_val_0, R_TCSR0_register['PWMA0']['bit_offset'])
R_temp_val_1 = set_bit(R_temp_val_1, R_TCSR1_register['PWMA1']['bit_offset'])

# The GenerateOut signals must be enabled in the TCSR (bit GENT set to 1). The PWM0
# signal is generated from the GenerateOut signals of Timer 0 and Timer 1, so these
# signals must be enabled in both timer/counters
R_temp_val_0 = set_bit(R_temp_val_0, R_TCSR0_register['GENT0']['bit_offset'])
R_temp_val_1 = set_bit(R_temp_val_1, R_TCSR1_register['GENT1']['bit_offset'])

# The counter can be set to count up or down. UDT
R_temp_val_0 = set_bit(R_temp_val_0, R_TCSR0_register['UDT0']['bit_offset'])
R_temp_val_1 = set_bit(R_temp_val_1, R_TCSR1_register['UDT1']['bit_offset'])

# set Autoreload (ARHT0 = 1)
R_temp_val_0 = set_bit(R_temp_val_0, R_TCSR0_register['ARHT0']['bit_offset'])
R_temp_val_1 = set_bit(R_temp_val_1, R_TCSR1_register['ARHT1']['bit_offset'])

# enable timer (ENT0 = 1)
R_temp_val_0 = set_bit(R_temp_val_0, R_TCSR0_register['ENT0']['bit_offset'])
R_temp_val_1 = set_bit(R_temp_val_1, R_TCSR1_register['ENT1']['bit_offset'])

In [12]:
L_TCSR0 = motor_control.ip_dict['axi_timer_1']['registers']['TCSR0']
L_TCSR1 = motor_control.ip_dict['axi_timer_1']['registers']['TCSR1']
L_TCSR0_address = L_TCSR0['address_offset']
L_TCSR1_address = L_TCSR1['address_offset']
L_TCSR0_register = L_TCSR0['fields'] # bit_offset for address
L_TCSR1_register = L_TCSR1['fields']
L_TLR0 = motor_control.ip_dict['axi_timer_1']['registers']['TLR0']
L_TLR1 = motor_control.ip_dict['axi_timer_1']['registers']['TLR1']
L_TLR0_address = L_TLR0['address_offset']
L_TLR1_address = L_TLR1['address_offset']

# create the configuration values for the control register
L_temp_val_0 = 0
L_temp_val_1 = 0

# The PWMA0 bit in TCSR0 and PWMB0 bit in TCSR1 must be set to 1 to enable PWM mode.
L_temp_val_0 = set_bit(L_temp_val_0, L_TCSR0_register['PWMA0']['bit_offset'])
L_temp_val_1 = set_bit(L_temp_val_1, L_TCSR1_register['PWMA1']['bit_offset'])

# The GenerateOut signals must be enabled in the TCSR (bit GENT set to 1). The PWM0
# signal is generated from the GenerateOut signals of Timer 0 and Timer 1, so these
# signals must be enabled in both timer/counters
L_temp_val_0 = set_bit(L_temp_val_0, L_TCSR0_register['GENT0']['bit_offset'])
L_temp_val_1 = set_bit(L_temp_val_1, L_TCSR1_register['GENT1']['bit_offset'])

# The counter can be set to count up or down. UDT
L_temp_val_0 = set_bit(L_temp_val_0, L_TCSR0_register['UDT0']['bit_offset'])
L_temp_val_1 = set_bit(L_temp_val_1, L_TCSR1_register['UDT1']['bit_offset'])

# set Autoreload (ARHT0 = 1)
L_temp_val_0 = set_bit(L_temp_val_0, L_TCSR0_register['ARHT0']['bit_offset'])
L_temp_val_1 = set_bit(L_temp_val_1, L_TCSR1_register['ARHT1']['bit_offset'])

# enable timer (ENT0 = 1)
L_temp_val_0 = set_bit(L_temp_val_0, L_TCSR0_register['ENT0']['bit_offset'])
L_temp_val_1 = set_bit(L_temp_val_1, L_TCSR1_register['ENT1']['bit_offset']) 

In [ ]:
# GPIO SETUP (Using MMIO)......
motor_enable_address = motor_control.ip_dict['axi_gpio_0']['phys_addr']
motor_enable = MMIO(motor_enable_address, 8)
motor_enable.write(0x4, 0x0) # Write to tri-state register to configure IO as OUTPUT

print("Program running")

time.sleep(10)

videoIn = cv2.VideoCapture(0)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w)
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h)
print("capture device is open: " + str(videoIn.isOpened()))

frame_queue = queue.Queue(maxsize=1)

capture_thread = threading.Thread(target=capture_frames, args=(videoIn, frame_queue))
capture_thread.daemon = True
capture_thread.start()

Program running


[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (616) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


capture device is open: True


In [ ]:
dt = 0.01                  
duration = 10              
t = np.arange(0, duration, dt)

#| Behavior            | Change This | Direction |
#| ------------------- | ----------- | --------- |
#| Slow response       | Kp          | Increase  |
#| Steady-state error  | Ki          | Increase  |
#| Overshoot           | Kd          | Increase  |
#| Oscillations        | Kp or Ki    | Decrease  |
#| Sluggish with noise | Kd          | Decrease  |

Kp = 11 
#Kp = 3
Ki = 0
Kd = 0

#Ki = 0.5

#Kd = 0.1
#Kd = 0.5
#total_gain = 0.009
total_gain = 0.1
integral = 0
prev_error = 0
base_power = 0

y = 0  # right error
r_power = 30
l_power = 30


#time.sleep(15)
original = -90
error = -90
    
l_forward = True
r_forward = True

err = []
r_val = []
l_val = []
corrections = []
original_error = [];
pv = [];

curr_time = 0
prev_time = time.time();

_period_ = 10000000 # 50Hz, 20ms

frame_list = []

#while (True):
#for i in range(1, 500000): 
for i in range(1, 20): 
    #if frame_queue.empty():
        #continue    
    while (frame_queue.empty()):
        time.sleep(0.15)
        
    #print(time.time())
        
    frame_vga = frame_queue.get()
    frame_list.append(frame_vga)
    print(frame_vga.shape)
    #frame_vga = frame_queue.queue[0]
    
    #test_image = cv2.cvtColor(frame_vga.reshape(frame_in_h, frame_in_w, 3).astype(np.uint8), cv2.COLOR_BGR2RGB)

    #plt.figure(figsize=(10,40))
    #plt.subplot(121)
    #plt.imshow(test_image)
    #plt.title('Original Image')
    #plt.xticks([])
    #plt.yticks([])
    
    b_thresholded = threshold(frame_vga)
    #threshold_list.append(b_thresholded)
    binary_warped = cv2.warpPerspective(b_thresholded,M, (frame_in_w, frame_in_h))[y_top:y_bottom, :]
    kernel = np.ones((20, 20), np.uint8)
    
     # Clean small blobs
    binary_cleaned = cv2.morphologyEx(binary_warped, cv2.MORPH_OPEN, kernel)

    # Fill small gaps
    binary_cleaned = cv2.morphologyEx(binary_cleaned, cv2.MORPH_CLOSE, kernel)
    
    leftx, lefty, rightx, righty = detect_lane_lines_connected_components(binary_cleaned)
    
    if leftx.size > 0 and lefty.size > 0 and rightx.size > 0 and righty.size > 0:
        # Fit a second order polynomial to each
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)

        angle = compute_lane_heading_angle(left_fit=left_fit, right_fit=right_fit, y_eval=y_eval)
    elif leftx.size > 0 and lefty.size > 0:
        # Fit a second order polynomial to each
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = [0,0,0]

        angle = compute_lane_heading_angle(left_fit=left_fit, y_eval=y_eval)
    elif rightx.size > 0 and righty.size > 0:
        # Fit a second order polynomial to each
        left_fit = [0,0,0]
        right_fit = np.polyfit(righty, rightx, 2)

        angle = compute_lane_heading_angle(right_fit=right_fit, y_eval=y_eval)
        
    #angle = angle * 1.5

    print("Steering angle: " + str(angle))
    
    curr_time = time.time()
    dt = curr_time - prev_time
    
    #error = angle - y
    error = angle
    
    #Proportional
    P = Kp * error

    #Integral 
    integral += error * dt
    I = Ki * integral

    #Derivative
    #derivative = (error - prev_error) / dt
    derivative = error / dt
    D = Kd * derivative

    correction = total_gain * (P + I + D)

    y += dt * correction 
    
    #May need average power
    r_power = 25 - correction
    l_power =  25 + correction
    
    #Keep between 0 and 100
    l_power = max(1, min(99, l_power))
    r_power = max(1, min(99, r_power))
    
    print("Correction: " + str(correction))
    print("r_power: " + str(r_power))
    print("l_power: " + str(l_power))
    print("Error: " + str(error))
    print("p: " + str(P))
    print("i: " + str(I))
    print("d: " + str(D))
    print("dt: " + str(dt))
    
    err.append(error) 
    r_val.append(r_power)
    l_val.append(l_power) 
    corrections.append(correction)
    original_error.append(original)
    pv.append(y)
    
    prev_error = error
    prev_time = curr_time
    
    period = int((_period_ & 0x0ffff) *100);
    r_pulse = int((int(r_power) & 0x07f)*period/100);
    l_pulse = int((int(l_power) & 0x07f)*period/100);
    
    control_motors(motor_enable, l_forward, r_forward)

    # Set period and duty cycle
    right_motor.write(R_TLR0_address, period)
    right_motor.write(R_TLR1_address, r_pulse)
    left_motor.write(L_TLR0_address, period)
    left_motor.write(L_TLR1_address, l_pulse)
    
    # Define the LOAD bits (1 << bit_offset)
    R_LOAD0 = 1 << R_TCSR0_register['LOAD0']['bit_offset']
    R_LOAD1 = 1 << R_TCSR1_register['LOAD1']['bit_offset']
    L_LOAD0 = 1 << L_TCSR0_register['LOAD0']['bit_offset']
    L_LOAD1 = 1 << L_TCSR1_register['LOAD1']['bit_offset']

    # Pulse LOAD bits
    right_motor.write(R_TCSR0_address, R_temp_val_0 | R_LOAD0)
    right_motor.write(R_TCSR1_address, R_temp_val_1 | R_LOAD1)
    left_motor.write(L_TCSR0_address, L_temp_val_0 | L_LOAD0)
    left_motor.write(L_TCSR1_address, L_temp_val_1 | L_LOAD1)

    # Re-enable timer (in case LOAD cleared ENT bits)
    right_motor.write(R_TCSR0_address, R_temp_val_0)
    right_motor.write(R_TCSR1_address, R_temp_val_1)
    left_motor.write(L_TCSR0_address, L_temp_val_0)
    left_motor.write(L_TCSR1_address, L_temp_val_1)

    time.sleep(0.1)  # Prevent busy loop (optional)
    


(480, 640, 3)
Steering angle: 0
Correction: 0.0
r_power: 25.0
l_power: 25.0
Error: 0
p: 0
i: 0.0
d: 0.0
dt: 1.2781014442443848
(480, 640, 3)
[8.991709352932833, 0.45626301676437103]
Steering angle: 4.723986184848602
Correction: 5.196384803333463
r_power: 19.803615196666538
l_power: 30.196384803333462
Error: 4.723986184848602
p: 51.96384803333463
i: 0.0
d: 0.0
dt: 0.7303941249847412
(480, 640, 3)
[8.06849132526408]
Steering angle: 8.06849132526408
Correction: 8.87534045779049
r_power: 16.12465954220951
l_power: 33.875340457790486
Error: 8.06849132526408
p: 88.75340457790489
i: 0.0
d: 0.0
dt: 0.7114560604095459
(480, 640, 3)
[22.310446122589987]
Steering angle: 22.310446122589987
Correction: 24.54149073484899
r_power: 1
l_power: 49.54149073484899
Error: 22.310446122589987
p: 245.41490734848986
i: 0.0
d: 0.0
dt: 0.7692227363586426
(480, 640, 3)
[53.686541544200935]
Steering angle: 53.686541544200935
Correction: 59.05519569862103
r_power: 1
l_power: 84.05519569862102
Error: 53.686541544200

In [15]:
for i, frame_vga in enumerate(frame_list):
    frame = frame_vga.reshape(frame_in_h, frame_in_w, 3).astype(np.uint8)
    
    # Save as PNG
    filename = f"test_images/frame_{i:04d}.jpg" 
    cv2.imwrite(filename, frame)